In [0]:
from google.colab import drive
import io
import os

import numpy as np
import matplotlib.pyplot as plt

drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


# **1. Calculate Mean and Covariance**

In [0]:
# Read input prices
pricesFile = open('russell_prices.txt', "r")
pricesFile=pricesFile.readlines()
prices = []

for line in pricesFile[2:]:
    prices.append([float(x) for x in line.split()])

# Turn to numpy
prices = np.array(prices)
# Calculate mean returns
returns = np.diff(prices)/prices[:,:-1]
meanReturns = np.mean(returns, 1)
 
# Calculate covariance
covarianceMatrix = np.cov(returns)


# **2. Step 1: Achieve Feasibility**

In [0]:
# Set constraints
u=np.ones(len(returns))
l=np.zeros(len(returns))

# Initialize algorithm
def initialize():

  # If constraints cause infeasibility, halt
  if np.sum(u)<1 or np.sum(l)>1:
    print("Infeasible")
    return
    
  # Initialize weights
  x=np.zeros(len(returns))
  
  # Iterate through weights to achieve feasibility
  for i in range(len(x)):
    max_increase = u[i] - x[i]
    total = np.sum(x)
    # If max_increase keeps sum below 1, add whole increase
    if total + max_increase <= 1:
      x[i] += max_increase
    # Otherwise, only add as much as possible
    else:
      x[i] += (1-total)
    # Stop if the last increase reached feasibility
    if np.sum(x) == 1:
      break
  return x


In [0]:
lamda = 1

# Improve initial guess
def improve(x, lamda):
  # Loop through every possible pair of assets
  # Each switches the optimal weight, so iterating every pair will give optimal
  for i in range(0,len(x)-1):
    for j in range(i+1,len(x)):
      # Calculate a and b constants
      # The caalculation of b1 combines the stray terms into the sum, which is equivalent
      a = lamda * (covarianceMatrix[i][i]+covarianceMatrix[j][j]-2*covarianceMatrix[i][j])
      b1 = np.dot(x, (covarianceMatrix[j,:] - covarianceMatrix[i,:]))
      
      b = 2*lamda*b1+(meanReturns[i]-meanReturns[j])
      # Special case if a is zero
      if a == 0:
        # Choose between moving all weight from one asset or the other
        if b >= 0:
          epsilon = -1
        else:
          epsilon = 1
      # Otherwise, psilon is calculated normally
      else:
        epsilon = -b/(2*a)
        
      # Calculate true epsilon within feasibility bounds
      epsilon_plus = min(x[i] - l[i], u[j] - x[j])
      epsilon_minus = min(u[i] - x[i], x[j] - l[j])
      if epsilon < -epsilon_minus:
        epsilon = -epsilon_minus
      elif epsilon > epsilon_plus:
        epsilon = epsilon_plus
      # Shift weight
      x[i] = x[i] - epsilon
      x[j] = x[j] + epsilon
  return x
